# Convertible Bond Pricer Using QuantLib


In [73]:
# !pip install QuantLib

In [74]:
import pandas as pd

In [75]:
import QuantLib as ql
print(ql.__version__)

1.32


In [76]:
calculation_date = ql.Date(22,2,2022)
ql.Settings.instance().evaluationDate = calculation_date

In [77]:
Curve_Data = pd.DataFrame({
    'Term': ['3 M', '12 M', '2 Y', '3 Y', '4 Y', '5 Y', '6 Y', '7 Y', '8 Y', '9 Y', '10 Y', '11 Y', '12 Y', '15 Y', '20 Y', '25 Y', '30 Y'],
    'Market Rate': [4.90943, 5.62014, 5.01423, 4.61500, 4.37235, 4.21810, 4.11853, 4.04790, 3.99590, 3.96250, 3.93995, 3.92100, 3.90770, 3.87780, 3.80137, 3.67615, 3.55655]
})

# Convert 'Market Rate' from percentage to decimal
Curve_Data['Market Rate'] = Curve_Data['Market Rate'] / 100.0

# Create a list of SimpleQuotes from the rates
rates = [ql.SimpleQuote(rate) for rate in Curve_Data['Market Rate']]

# Create a list of Periods from the terms
tenors = [ql.Period(term) for term in Curve_Data['Term']]

# Other required QuantLib objects
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
fixed_leg_frequency = ql.Annual
fixed_leg_convention = ql.ModifiedFollowing
fixed_leg_day_counter = ql.Thirty360(ql.Thirty360.USA)
index = ql.USDLibor(ql.Period('3M'))

# Create a list of SwapRateHelpers
rate_helpers = [
    ql.SwapRateHelper(ql.QuoteHandle(rate), tenor, calendar, fixed_leg_frequency, fixed_leg_convention, fixed_leg_day_counter, index)
    for rate, tenor in zip(rates, tenors)
]

# Now we construct the yield curve
curve = ql.PiecewiseFlatForward(0, ql.TARGET(), rate_helpers, ql.Actual360())

In [78]:
Curve_Data_New = Curve_Data.copy()
Curve_Data_New['Market Rate'] = Curve_Data['Market Rate'] + 0.0001

# Create a list of SimpleQuotes from the rates
rates_new = [ql.SimpleQuote(rate) for rate in Curve_Data_New['Market Rate']]

# Create a list of Periods from the terms
tenors_new = [ql.Period(term) for term in Curve_Data_New['Term']]

# Create a list of SwapRateHelpers
rate_helpers_new = [
    ql.SwapRateHelper(ql.QuoteHandle(rate), tenor, calendar, fixed_leg_frequency, fixed_leg_convention, fixed_leg_day_counter, index)
    for rate, tenor in zip(rates_new, tenors_new)
]

# Now we construct the yield curve
curve_new = ql.PiecewiseFlatForward(0, ql.TARGET(), rate_helpers_new, ql.Actual360())

In [79]:
# Convert the swap curve to a YieldTermStructureHandle
yield_curve_handle = ql.YieldTermStructureHandle(curve)
yield_curve_handle_new = ql.YieldTermStructureHandle(curve_new)

In [80]:
# St. Mary Land & Exploration Company 
# Bloomberg ticker: SM 5.75 03/15/22 

redemption = 100.00
face_amount = 100.0
spot_price = 3.02
conversion_price = 7.068
conversion_ratio = 14.14827

issue_date = ql.Date(30,1,2013)        
maturity_date = ql.Date(1,2,2033)

settlement_days = 2
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
coupon = 0.03
frequency = ql.Semiannual
tenor = ql.Period(frequency)

day_count = ql.Thirty360(ql.Thirty360.BondBasis)
accrual_convention = ql.Unadjusted
payment_convention = ql.Unadjusted

call_dates = [ql.Date(1,2,2019)]
call_price = 100.0
put_dates = [ql.Date(1,2,2019), ql.Date(1,2,2023), ql.Date(1,2,2028)]
put_price = 100.0

# assumptions
dividend_yield = 0
credit_spread_rate = 0.04
risk_free_rate = 0.04

volatility = 0.30

In [81]:
callability_schedule = ql.CallabilitySchedule()


for call_date in call_dates:
   callability_price  = ql.BondPrice(call_price, 
                                            ql.BondPrice.Clean)
   callability_schedule.append(ql.Callability(callability_price, 
                                       ql.Callability.Call,
                                       call_date)
                        )
    
for put_date in put_dates:
    puttability_price = ql.BondPrice(put_price, 
                                            ql.BondPrice.Clean)
    callability_schedule.append(ql.Callability(puttability_price,
                                               ql.Callability.Put,
                                               put_date))

In [82]:
dividend_schedule = ql.DividendSchedule() # No dividends
dividend_amount = dividend_yield*spot_price
next_dividend_date = ql.Date(1,12,2004)
dividend_amount = spot_price*dividend_yield
for i in range(4):
    date = calendar.advance(next_dividend_date, 1, ql.Years)
    dividend_schedule.append(
        ql.FixedDividend(dividend_amount, date)
    )

In [83]:
schedule = ql.Schedule(issue_date, maturity_date, tenor,
                       calendar, accrual_convention, accrual_convention,
                       ql.DateGeneration.Backward, False)

credit_spread_handle = ql.QuoteHandle(ql.SimpleQuote(credit_spread_rate))
exercise = ql.AmericanExercise(calculation_date, maturity_date)

convertible_bond = ql.ConvertibleFixedCouponBond(exercise,
                                                     conversion_ratio,
                                                     callability_schedule, 
                                                     issue_date,
                                                     settlement_days,
                                                     [coupon],
                                                     day_count,
                                                     schedule,
                                                     redemption)

In [84]:
spot_price_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
# yield_ts_handle = ql.YieldTermStructureHandle(
#     ql.FlatForward(calculation_date, risk_free_rate, day_count)
# )
dividend_ts_handle = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_yield, day_count)
)
volatility_ts_handle = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar,volatility, day_count)
)

bsm_process = ql.BlackScholesMertonProcess(spot_price_handle, 
                                           dividend_ts_handle,
                                           yield_curve_handle, # yield_ts_handle,
                                           volatility_ts_handle)

In [85]:
time_steps = 1000
engine = ql.BinomialConvertibleEngine(bsm_process, "crr", time_steps, credit_spread_handle, dividend_schedule)

In [86]:
convertible_bond.setPricingEngine(engine)
NPV = convertible_bond.NPV()
print ("NPV ", NPV)
print ("Accrued ", convertible_bond.accruedAmount())

NPV  96.71797007561301
Accrued  0.19166666666665666


# Greeks Calculation

In [87]:
delta_spot_price = 0.01* spot_price
spot_price_handle_up = ql.QuoteHandle(ql.SimpleQuote(spot_price+delta_spot_price))
bsm_process_up = ql.BlackScholesMertonProcess(spot_price_handle_up, 
                                           dividend_ts_handle,
                                           yield_curve_handle,
                                           volatility_ts_handle)
engine_up = ql.BinomialConvertibleEngine(bsm_process_up, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_up)
NPV_up = convertible_bond.NPV()
print ("NPV_up ", NPV_up)

NPV_up  96.73872732133808


In [88]:
spot_price_handle_down = ql.QuoteHandle(ql.SimpleQuote(spot_price-delta_spot_price))
bsm_process_down = ql.BlackScholesMertonProcess(spot_price_handle_down, 
                                           dividend_ts_handle,
                                           yield_curve_handle,
                                           volatility_ts_handle)
engine_down = ql.BinomialConvertibleEngine(bsm_process_down, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_down)
NPV_down = convertible_bond.NPV()
print ("NPV_down ", NPV_down)

NPV_down  96.697212829888


In [89]:
parity = 42.7278

In [90]:
delta = (NPV_up - NPV_down)/(0.02*parity)
delta

0.04858018836690972

In [91]:
# Calculate Gamma

# Re-calculate for a further 1% increase
spot_price_handle_up2 = ql.QuoteHandle(ql.SimpleQuote(spot_price + 2*delta_spot_price))
bsm_process_up2 = ql.BlackScholesMertonProcess(spot_price_handle_up2, dividend_ts_handle, yield_curve_handle, volatility_ts_handle)
engine_up2 = ql.BinomialConvertibleEngine(bsm_process_up2, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_up2)
NPV_up2 = convertible_bond.NPV()

# Delta for the up 2% move
delta_up = (NPV_up2 - NPV_up) / (0.01 * parity)

# Calculate Gamma (change in delta for a 1% stock move)
gamma = (delta_up - delta) / (0.01 * parity)
gamma

0.034703103832992216

In [92]:
# Calculate Vega
volatility_ts_handle_vup = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar,volatility*1.01, day_count)
)
bsm_process_vup = ql.BlackScholesMertonProcess(spot_price_handle, 
                                           dividend_ts_handle,
                                           yield_curve_handle,
                                           volatility_ts_handle_vup)
engine_vup = ql.BinomialConvertibleEngine(bsm_process_vup, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_vup)
NPV_vup = convertible_bond.NPV()

volatility_ts_handle_vdown = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar,volatility*0.99, day_count)
)
bsm_process_vdown = ql.BlackScholesMertonProcess(spot_price_handle,
                                             dividend_ts_handle,
                                             yield_curve_handle,
                                             volatility_ts_handle_vdown)
engine_vdown = ql.BinomialConvertibleEngine(bsm_process_vdown, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_vdown)
NPV_vdown = convertible_bond.NPV()

vol_change = volatility*0.01
#vega = (NPV_vup - NPV_vdown) / (2 * vol_change)
vega1 = (NPV_vup - NPV_vdown) / 2
vega1

0.007870346364647673

In [93]:
# Calculate Rho
bsm_process_yup = ql.BlackScholesMertonProcess(spot_price_handle, 
                                           dividend_ts_handle,
                                           yield_curve_handle_new,
                                           volatility_ts_handle)
engine_yup = ql.BinomialConvertibleEngine(bsm_process_yup, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine_yup)
NPV_yup = convertible_bond.NPV()
rho = 100*(NPV_yup - NPV)
rho

-0.8340039436518509

In [94]:
# Calculate Theta
ql.Settings.instance().evaluationDate = calculation_date + 1

engine = ql.BinomialConvertibleEngine(bsm_process, "crr", time_steps, credit_spread_handle, dividend_schedule)
convertible_bond.setPricingEngine(engine)
NPV_tomorrow = convertible_bond.NPV()
theta = (NPV_tomorrow - NPV)*360
theta

24.858391384887

In [95]:
# Create a table of the prices, accrued, and the Greeks
output = pd.DataFrame({
    'Price': [NPV],
    'Accrued': [convertible_bond.accruedAmount()],
    'Delta': [delta],
    'Gamma': [gamma],
    'Vega': [vega1],
    'Rho': [rho],
    'Theta': [theta]
})

output

,Price,Accrued,Delta,Gamma,Vega,Rho,Theta
0,96.71797,0.2,0.04858,0.034703,0.00787,-0.834004,24.858391


In [96]:
output.to_excel("output.xlsx")